# Structured Output with Google Gemini

In order to build reliable pipelines in which LLMs consistently return outputs in the same format, we are using a **Structured Output**
- This means that we define a blueprint for the output
- We pass the 'blueprint' to the LLM
- Then the LLM output will conform to the blueprint.

This 'blueprint' in the LLM jargon is often called a "schema".

**To generate structured outputs with Google Gemini, we'll use the library `instructor`**.
This will make it very easy to switch between different model providers.

## Setup Google Gemini API

In [1]:
!pip install instructor --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.5 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai
import os
from pydantic import BaseModel
from enum import Enum
import instructor

In [3]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [4]:
genai.configure(api_key=GOOGLE_API_KEY)

## Generate JSON
When the model is configured to output JSON, it responds to any prompt with JSON-formatted output.

You can control the structure of the JSON response by supplying a schema. There are two ways to supply a schema to the model:

- As text in the prompt
- As a structured schema supplied through model configuration

Both approaches work for Gemini 1.5 Flash and for Gemini 1.5 Pro.

#### Example

We will configure Gemini to answer math questions.
- It will provide step-by-step explainations and intermediate results
- It will finish by answering with the final answer.

In [6]:
# Each step is some explanation and an intermediate output
class Step(BaseModel):
    explanation: str
    output: str

# The complete problem-solving reasoning is made of several steps
#  leading to a final answer.
class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

In [7]:
google_client = genai.GenerativeModel(
    model_name="gemini-1.5-flash")

client = instructor.from_gemini(
    client=google_client,
    mode=instructor.Mode.GEMINI_JSON,
)

In [8]:
# note that client.chat.completions.create will also work
response = client.messages.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful math tutor. \
                        Guide the user through the solution step by step."},
        {
            "role": "user",
            "content": "how can I solve 8x + 7 = -23"},
    ],
    response_model=MathReasoning,
)

In [18]:
# @title
from pprint import pprint
for step in response.steps:
    print(step.explanation)
    print(step.output)
print(f"\n\nfinal answer: {response.final_answer}")

Subtract 7 from both sides of the equation to isolate the term with 'x'.
8x + 7 - 7 = -23 - 7
8x = -30
Divide both sides of the equation by 8 to solve for 'x'.
8x / 8 = -30 / 8
x = -30/8
Simplify the fraction by dividing both numerator and denominator by their greatest common divisor, which is 2.
x = -15/4


final answer: x = -15/4


Learn more about instructor here: https://python.useinstructor.com/